# Parallelization and Security Demo

## Setup

1. Install [`elan`](https://github.com/leanprover/elan)
2. Install [Mathlib4](https://github.com/leanprover-community/mathlib4/tree/v4.20.1)
3. Enter the Mathlib directory, and run `lake exe cache get` (otherwise it will be compiled from scratch! takes hours)

In [1]:
from pantograph import Server
from pantograph.expr import TacticExpr

import os

DIR_MATHLIB = os.path.expanduser("~/Projects/contrib/formalization/mathlib4")

## Background

Each Pantograph instance carries with it an Environment. We can inspect the type and location of a symbol in the environment.

In [2]:
server = await Server.create()
await server.env_inspect_async("Nat.add_comm")

{'type': {'pp': '∀ (n m : Nat), n + m = m + n'},
 'sourceStart': {'line': 156, 'column': 0},
 'sourceEnd': {'line': 161, 'column': 14},
 'module': 'Init.Data.Nat.Basic',
 'isUnsafe': False}

We can run a basic form proof verification using `check_compile`. Note that this version is insecure.

In [3]:
code = """
example (n : Nat) : n = n := by
  rfl
"""
await server.check_compile_async(code)

[CompilationUnit(i_begin=0, i_end=39, messages=[], invocations=None, goal_state=None, goal_src_boundaries=None, new_constants=None)]

Proof search and verification use the same interface, which has been deliberately engineered for security. In the example below, the agent supplying the proof has no chance of altering the statement.

In [4]:
state = await server.goal_start_async("forall (n : Nat), n = n")
proof_body = """
by
   intro n
   rfl
"""
state = await server.goal_tactic_async(state, TacticExpr(proof_body))
state

GoalState(state_id=1, goals=[], messages=[], _sentinel=[0])

We can also run a proof incrementally. This forms the basis of tree search.

In [5]:
state = await server.goal_start_async("forall (n : Nat), n = n")
state = await server.goal_tactic_async(state, "intro n")
state = await server.goal_tactic_async(state, "rfl")
state

GoalState(state_id=4, goals=[], messages=[], _sentinel=[0, 2, 3])

## Serialization

Pantograph supports serialization/deserialization of goal states.

In [6]:
server = await Server.create(imports=["Mathlib"], project_path=DIR_MATHLIB)

We can save and load goal states.

In [7]:
state = await server.goal_start_async("forall (n : Nat), n = n")
await server.goal_save_async(state, "/tmp/goal.olean")

In [8]:
server2 = await Server.create(imports=["Mathlib"], project_path=DIR_MATHLIB)

Warning: Due to the possibilities of constant initializers, serializing and deserializing environment is only permitted under special circumstances.

In [9]:
state2 = await server2.goal_load_async("/tmp/goal.olean")
print(state2)


⊢ ∀ (n : ℕ), n = n


## Security

Even just type checking top-level Lean code is a security vulnerability!

In [10]:
code = """
#eval do
  let result <- IO.Process.output { cmd := "ls" }
  IO.println s!"{result.stdout}"
"""
await server.check_compile_async(code)

[CompilationUnit(i_begin=0, i_end=93, messages=['\x1bArchive\x1b[m\x1b[m\nArchive.lean\nbors.toml\n\x1bCache\x1b[m\x1b[m\nCODE_OF_CONDUCT.md\n\x1bCounterexamples\x1b[m\x1b[m\nCounterexamples.lean\n\x1bdocs\x1b[m\x1b[m\ndocs.lean\n\x1bDownstreamTest\x1b[m\x1b[m\n\x1bExamples\x1b[m\x1b[m\nGNUmakefile\nlake-manifest.json\nlakefile.lean\nlean-toolchain\nLICENSE\n\x1bLongestPole\x1b[m\x1b[m\n\x1bMathlib\x1b[m\x1b[m\nMathlib.lean\n\x1bMathlibTest\x1b[m\x1b[m\nREADME.md\n\x1bSandbox\x1b[m\x1b[m\n\x1bscripts\x1b[m\x1b[m\n\x1bShake\x1b[m\x1b[m\n\x1bwidget\x1b[m\x1b[m\n\n'], invocations=None, goal_state=None, goal_src_boundaries=None, new_constants=None)]